<a href="https://colab.research.google.com/github/steph128/BERT-for-Biomedical-Texts-Named-Entity-Recognition-/blob/main/Train_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
! pip install transformers
! pip install pytorch_pretrained_bert
! pip install seqeval

In [24]:
import pandas as pd
import csv
import os
import numpy as np
from pytorch_pretrained_bert import BertModel, BertTokenizer, BertConfig

from transformers import BertForTokenClassification, AdamW
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, random_split
from torch.utils.data import RandomSampler, SequentialSampler

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from transformers import get_linear_schedule_with_warmup

from tqdm import tqdm, trange
from seqeval.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)


'Tesla P100-PCIE-16GB'

In [26]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [27]:
from nerdata import get_sentences, get_unique_labels, tokenize_and_preserve_labels,padding, preprocess, create_optimizer, create_scheduler, training

In [28]:
main_dir = "/content/gdrive/MyDrive/NLP"
datasets = ['BC5CDR-chem-IOBES', 'BC5CDR-disease-IOBES', 'JNLPBA-IOBES', 'BC4CHEMD-IOBES', 'NCBI-disease-IOBES']
train_file_paths = []
test_file_paths = []
for f in datasets:
    train_file_paths.append(os.path.join(main_dir, f, 'train.tsv'))
    train_file_paths.append(os.path.join(main_dir, f, 'devel.tsv'))
    test_file_paths.append(os.path.join(main_dir, f, "test.tsv"))

In [29]:
import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [30]:
train_lists = []
for i in range(len(train_file_paths)):
    open_it = open(train_file_paths[i])
    read_tsv = csv.reader(open_it, delimiter="\t")
    for row in read_tsv:
        train_lists.append(row)
len(train_lists)

2103131

In [31]:
test_lists = []
for i in range(len(test_file_paths)):
    open_it = open(test_file_paths[i])
    read_tsv = csv.reader(open_it, delimiter="\t")
    for row in read_tsv:
        test_lists.append(row)
len(test_lists)

789219

In [32]:
df_train = pd.DataFrame(train_lists, columns=["tokens","labels"])
df_train = df_train.replace('None',np.nan)
df_test = pd.DataFrame(test_lists, columns=["tokens","labels"])
df_test = df_test.replace('None',np.nan)

df_train = df_train.dropna()
df_test = df_test.dropna()

In [33]:
train_sentences, train_labels = get_sentences(df_train)
print(len(train_sentences),len(train_labels))

86383 86383


In [40]:
unique_labels,id_to_label,label_to_id = get_unique_labels(df_train)
unique_labels

['S-Chemical',
 'O',
 'B-Chemical',
 'I-Chemical',
 'E-Chemical',
 'B-Disease',
 'E-Disease',
 'I-Disease',
 'S-Disease',
 'B-DNA',
 'E-DNA',
 'B-protein',
 'E-protein',
 'S-protein',
 'I-protein',
 'B-cell_type',
 'I-cell_type',
 'E-cell_type',
 'I-DNA',
 'S-cell_type',
 'S-DNA',
 'S-cell_line',
 'B-cell_line',
 'E-cell_line',
 'I-cell_line',
 'B-RNA',
 'I-RNA',
 'E-RNA',
 'S-RNA',
 'PAD']

In [35]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

In [36]:
train_tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(tokenizer,sent, labs)
    for sent, labs in zip(train_sentences, train_labels)
]

In [37]:
train_tokenized_texts = [token_label[0] for token_label in train_tokenized_texts_and_labels]
train_labels = [token_label[1] for token_label in train_tokenized_texts_and_labels]

In [38]:
train_input, train_tags = padding(tokenizer,train_tokenized_texts, train_labels, label_to_id )

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (60365 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (98741 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (60271 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (99381 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (526802 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequen

In [39]:
train_dataloader, valid_dataloader = preprocess(train_input, train_tags, batch_size = 12)

In [41]:
model = BertForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(label_to_id),
    output_attentions = False,
    output_hidden_states = False
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [42]:
model.cuda();

In [45]:
optimizerr = create_optimizer(model)

In [49]:
epochs = 3
total_steps = len(train_dataloader) * epochs
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizerr,
    num_warmup_steps = 0,
    num_training_steps=total_steps
)

In [ ]:
loss_values, validation_loss_values = [],[] #store for each epoch
for _ in trange(epochs, desc = "Epoch"): # prefix message desc
  model.train()
  total_loss = 0

  for step,batch in enumerate(train_dataloader):
      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_labels = batch

      b_input_ids = b_input_ids.type(torch.LongTensor).to(device)
      b_input_mask = b_input_mask.type(torch.LongTensor).to(device)
      b_labels = b_labels.type(torch.LongTensor).to(device)

      model.zero_grad() # clear prev. accumulated gradients 

      outputs = model(b_input_ids, token_type_ids = None, attention_mask = b_input_mask, 
                    labels = b_labels)
      loss = outputs[0]
      loss.backward()
      total_loss += loss.item()
      # clip gradients to prevent exploding gradients
      torch.nn.utils.clip_grad_norm_(parameters = model.parameters(), max_norm = max_grad_norm)
      optimizer.step()
      scheduler.step() # updated learning rate

  # calculate average loss over the batches
  avg_train_loss = total_loss / len(train_dataloader)
  print("Average train loss:{}".format(avg_train_loss))

  loss_values.append(avg_train_loss)

  model.eval()
  eval_loss, eval_accuracy = 0,0
  nb_eval_steps, nb_eval_examples = 0,0

  predictions, true_labels = [],[]
  for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    b_input_ids = b_input_ids.type(torch.LongTensor).to(device)
    b_input_mask = b_input_mask.type(torch.LongTensor).to(device)
    b_labels = b_labels.type(torch.LongTensor).to(device)

    with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids = None, attention_mask = b_input_mask,
                      labels= b_labels)
      logits = outputs[1].detach().cpu().numpy() #(7, 100, 30) # 100 is max len, 30 is no. labels 
      label_ids = b_labels.to("cpu").numpy()

      eval_loss += outputs[0].mean().item() # add loss for this batch
      predictions.extend([list(p) for p in np.argmax(logits,axis=2)]) # 8639 for 100 each sentence (axis 2 is 30- labels prob)
      true_labels.extend(label_ids)

  eval_loss = eval_loss / len(valid_dataloader) # average of all the batches
  print("Validation loss:{}.format(eval_loss")
  validation_loss_values.append(eval_loss)

  # get actual labels (label_to_id) for predictions
  temp = []
  final_predictions = []
  for sent_no, pred in enumerate(predictions): 
    for i, p in enumerate(pred):
      if true_labels[sent_no][i] != 29:
       temp.append(unique_labels[p])
    final_predictions.append(temp)
    temp = []

# get actual labels for true labels
  real_labels = []
  temp2=[]
  for sent in true_labels:
    for lab in sent:
      if lab !=29:
        temp2.append(unique_labels[lab])
    real_labels.append(temp2)
    temp2=[]

  print("Validation Accuracy: {}".format(accuracy_score(real_labels,final_predictions)))
  print("Validation F1-Score: {}".format(f1_score(real_labels,final_predictions)))
  print("Precision: {}".format(precision_score(real_labels,final_predictions)))
  print("Recall: {}".format(recall_score(real_labels,final_predictions)))


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Average train loss:0.2780567337252139
Validation loss:{}.format(eval_loss
Validation Accuracy: 0.9302033939520261
Validation F1-Score: 0.763098298020146
Precision: 0.7377498388136686


Epoch:  33%|███▎      | 1/3 [32:44<1:05:28, 1964.44s/it]

Recall: 0.7902506402693448
Average train loss:0.15361311978610645
Validation loss:{}.format(eval_loss
Validation Accuracy: 0.9356226008719365
Validation F1-Score: 0.7881009825989675
Precision: 0.7412525354969574


Epoch:  67%|██████▋   | 2/3 [1:05:35<32:46, 1966.47s/it]

Recall: 0.8412707547984231
Average train loss:0.10556123711497654
Validation loss:{}.format(eval_loss
Validation Accuracy: 0.937786809789311
Validation F1-Score: 0.7937653182056013
Precision: 0.761029702970297


Epoch: 100%|██████████| 3/3 [1:38:25<00:00, 1968.46s/it]

Recall: 0.8294437570141866


In [ ]:
torch.save(model.state_dict(), '/content/gdrive/MyDrive/NLP/NERmodel.pt')

In [ ]:
model.load_state_dict(torch.load('/content/gdrive/MyDrive/NLP/NERmodel.pt'))

<All keys matched successfully>

In [ ]:
model.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis